# Create GeoJSONs for the Website
1. Create GeoJSON of state districts without states that have a single district only (i.e., no at-large districts).
2. Create GeoJSON of state outlines (just state borders, no districts).

In [1]:
%matplotlib inline

# Assuming we are in the notebooks directory, we need to move one up:
%cd ..

/home/olivia/GitHub/test/gerrymandering_and_computational_redistricting


In [2]:
from __future__ import print_function

import os
import us
import pandas as pd
import geopandas as gpd

import numpy as np

from shapely.geometry import shape, mapping
from utils.settings import clustered_geojson_results_dir, usa_shapefile_path, site_dir

In [3]:
# Load the map of the whole USA with every congressional district:
usa = gpd.read_file(usa_shapefile_path)

In [9]:
usa.head()

,ALAND,AWATER,CD115FP,CDSESSN,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAMELSAD,STATEFP,geometry
0,1477946266785,245390495931,00,115,N,0200,+63.3461909,-152.8370690,C1,G5200,Congressional District (at Large),02,"(POLYGON ((179.388742 51.941917, 179.404562 51..."
1,5047194742,1398720828,00,115,N,1000,+38.9985380,-075.4416616,C1,G5200,Congressional District (at Large),10,"POLYGON ((-75.789023 39.65979, -75.788922 39.6..."
2,376964956503,3866986696,00,115,N,3000,+47.0511771,-109.6348174,C1,G5200,Congressional District (at Large),30,"POLYGON ((-116.049116 48.000828, -116.049118 4..."
3,178711813026,4399093501,00,115,N,3800,+47.4421698,-100.4608163,C1,G5200,Congressional District (at Large),38,"POLYGON ((-104.048967 48.86953399999999, -104...."
4,196348407642,3380782733,00,115,N,4600,+44.4467957,-100.2381762,C1,G5200,Congressional District (at Large),46,"POLYGON ((-104.057879 44.997605, -104.050783 4..."


In [28]:
usa_no_al = usa[usa['NAMELSAD'].str.contains('Large') == False] # remove all at large

In [29]:
mini_usa = usa_no_al[['geometry', 'GEOID']]
mini_usa.head()

,geometry,GEOID
7,"POLYGON ((-88.46442999999999 31.697952, -88.46...",0101
8,"POLYGON ((-114.042377 35.72514, -114.041594 35...",0401
9,"POLYGON ((-92.95095099999999 35.740389, -92.95...",0501
10,"POLYGON ((-123.719174 41.595613, -123.719001 4...",0601
11,"POLYGON ((-105.145973 39.577416, -105.145601 3...",0801


In [41]:
# Now we have gotten rid of the columns we do not need, we can round the coordinates:
for index, i in enumerate(mini_usa.index):
    geojson = mapping(mini_usa['geometry'].iloc[index])
    t = np.asarray(geojson['coordinates'])
    try: 
        t = np.round(t, 6)
    except TypeError:
        pass
    try:
        for i, points in enumerate(t):
            new_points = []
            for p in points[0]:
                p = list(p)
                p[0] = np.round(p[0], 6)
                p[1] = np.round(p[1], 6)
                p = tuple(p)
                new_points.append(p)
            points[0] = new_points
    except TypeError:
        for i, points in enumerate(t):
            new_points = []
            for p in points:
                p = list(p)
                p[0] = np.round(p[0], 6)
                p[1] = np.round(p[1], 6)
                p = tuple(p)
                new_points.append(p)
            points = new_points
    geojson['coordinates'] = list(t)
    mini_usa.iloc[index, mini_usa.columns.get_loc('geometry')] = shape(geojson)

0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [45]:
mini_usa.head()

,geometry,GEOID
7,"POLYGON ((-88.46442999999999 31.697952, -88.46...",0101
8,"POLYGON ((-114.042377 35.72514, -114.041594 35...",0401
9,"POLYGON ((-92.950951 35.740389, -92.950737 35....",0501
10,"POLYGON ((-123.719174 41.595613, -123.719001 4...",0601
11,"POLYGON ((-105.145973 39.577416, -105.145601 3...",0801


In [46]:
with open('./gerrymandering-site/json/geo/USA.geo.json', 'w') as outfile:
    outfile.write(mini_usa.to_json())

In [8]:
state_outlines = usa.dissolve(by = 'STATEFP')

In [11]:
state_outlines

,geometry,ALAND,AWATER,CD115FP,CDSESSN,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAMELSAD
STATEFP,,,,,,,,,,,,
01,"POLYGON ((-85.474024 34.344548, -85.47045 34.3...",1.571303e+10,2.275872e+09,01,115,N,0101,+31.0020519,-087.7879717,C2,G5200,Congressional District 1
02,"(POLYGON ((-179.230233 51.283927, -179.225686 ...",1.477946e+12,2.453905e+11,00,115,N,0200,+63.3461909,-152.8370690,C1,G5200,Congressional District (at Large)
04,"POLYGON ((-109.047918 32.426376, -109.047933 3...",1.425460e+11,3.540736e+08,01,115,N,0401,+34.9710133,-110.7259249,C2,G5200,Congressional District 1
05,"POLYGON ((-94.485877 33.637893, -94.4858579999...",5.003631e+10,1.233952e+09,01,115,N,0501,+35.2939086,-091.2583060,C2,G5200,Congressional District 1
06,"(POLYGON ((-119.636068 33.280713, -119.634795 ...",7.274826e+10,1.946597e+09,01,115,N,0601,+40.7319880,-121.5105160,C2,G5200,Congressional District 1
08,"POLYGON ((-104.943371 40.998213, -104.942761 4...",4.919853e+08,7.057884e+06,01,115,N,0801,+39.7322859,-104.9087582,C2,G5200,Congressional District 1
09,"POLYGON ((-72.05005799999999 41.276614, -72.05...",1.749273e+09,5.135420e+07,01,115,N,0901,+41.9284627,-073.0173833,C2,G5200,Congressional District 1
10,"POLYGON ((-75.789023 39.65979, -75.788922 39.6...",5.047195e+09,1.398721e+09,00,115,N,1000,+38.9985380,-075.4416616,C1,G5200,Congressional District (at Large)
11,POLYGON ((-77.11975099999999 38.93434999999999...,1.583650e+08,1.863340e+07,98,115,N,1198,+38.9041031,-077.0172290,C4,G5200,Delegate District (at Large)


In [14]:
state_outlines[['geometry']]

,geometry
STATEFP,
01,"POLYGON ((-85.474024 34.344548, -85.47045 34.3..."
02,"(POLYGON ((-179.230233 51.283927, -179.225686 ..."
04,"POLYGON ((-109.047918 32.426376, -109.047933 3..."
05,"POLYGON ((-94.485877 33.637893, -94.4858579999..."
06,"(POLYGON ((-119.636068 33.280713, -119.634795 ..."
08,"POLYGON ((-104.943371 40.998213, -104.942761 4..."
09,"POLYGON ((-72.05005799999999 41.276614, -72.05..."
10,"POLYGON ((-75.789023 39.65979, -75.788922 39.6..."
11,POLYGON ((-77.11975099999999 38.93434999999999...


In [16]:
state_outlines['STATEFP'] = state_outlines.index

In [18]:
state_outlines[['geometry', 'STATEFP']]

,geometry,STATEFP
STATEFP,,
01,"POLYGON ((-85.474024 34.344548, -85.47045 34.3...",01
02,"(POLYGON ((-179.230233 51.283927, -179.225686 ...",02
04,"POLYGON ((-109.047918 32.426376, -109.047933 3...",04
05,"POLYGON ((-94.485877 33.637893, -94.4858579999...",05
06,"(POLYGON ((-119.636068 33.280713, -119.634795 ...",06
08,"POLYGON ((-104.943371 40.998213, -104.942761 4...",08
09,"POLYGON ((-72.05005799999999 41.276614, -72.05...",09
10,"POLYGON ((-75.789023 39.65979, -75.788922 39.6...",10
11,POLYGON ((-77.11975099999999 38.93434999999999...,11


In [19]:
state_outlines = state_outlines[['geometry', 'STATEFP']]

In [23]:
with open(site_dir + 'json/geo/state_outlines.geo.json', 'w') as outfile:
    outfile.write(state_outlines.to_json())

In [ ]:
# Load the map of the whole USA with every congressional district:
districts = gpd.read_file(usa_shapefile_path)